In [ ]:
import dash
from dash import html, dcc
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
from dash_extensions.javascript import assign
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Polygon
import numpy as np
from datetime import datetime, time

from velib_spot_predictor.data.load_data import (
    load_station_information,
    load_prepared,
)
from velib_spot_predictor.data.geo import CatchmentAreaBuilderColumns

In [ ]:
def filter_time(df_with_datetime, hour, minute):
    return df_with_datetime[
        df_with_datetime["datetime"].dt.floor("min").dt.time
        == time(hour, minute)
    ]


# Join the availability and station information
def join_occupation_and_station_information(
    occupation_df, station_information_catchment_area
):
    station_occupation = station_information_catchment_area.merge(
        occupation_df[
            ["station_id", "num_bikes_available", "num_docks_available"]
        ],
        on="station_id",
        how="right",
    )
    station_occupation["occupation"] = (
        100
        * station_occupation["num_bikes_available"]
        / station_occupation["capacity"]
    )
    station_occupation["tooltip"] = station_occupation["name"]
    return station_occupation


def extract_hour_minute(time_str):
    return int(time_str.split(":")[0]), int(time_str.split(":")[1])

In [ ]:
initial_time = time(7, 40)

arrondissements = gpd.read_file("../data/external/arrondissements.geojson")
quartier_paris = gpd.read_file("../data/external/quartier_paris.geojson")
occupation_df = load_prepared("../data/interim/data_20230907.pkl")
station_information = load_station_information(
    "../data/raw/station_information.json"
)
station_catchment_area = (
    CatchmentAreaBuilderColumns(
        longitude="lon",
        latitude="lat",
    )
    .run(station_information)
    .set_crs("EPSG:4326")
)
occupation_df_time = filter_time(
    occupation_df, initial_time.hour, initial_time.minute
)
station_occupation = join_occupation_and_station_information(
    occupation_df_time,
    gpd.GeoDataFrame(
        station_information, geometry=station_catchment_area, crs="EPSG:4326"
    ),
)
station_information = gpd.GeoDataFrame(
    station_information,
    geometry=gpd.points_from_xy(
        station_information["lon"], station_information["lat"]
    ),
    crs="EPSG:4326",
)

In [ ]:
def get_arrondissements_with_occupation(arrondissements, occupation_df_time):
    arrondissements_with_occupation = (
        arrondissements.sjoin(
            gpd.GeoDataFrame(
                occupation_df_time[
                    ["capacity", "num_bikes_available", "num_docks_available"]
                ],
                geometry=gpd.points_from_xy(
                    occupation_df_time["lon"], occupation_df_time["lat"]
                ),
                crs="EPSG:4326",
            )
        )
        .groupby("geometry", as_index=False)
        .agg(
            {
                "c_ar": "first",
                "l_ar": "first",
                "capacity": "sum",
                "num_bikes_available": "sum",
                "num_docks_available": "sum",
            }
        )
    )
    arrondissements_with_occupation = gpd.GeoDataFrame(
        arrondissements_with_occupation,
        geometry=arrondissements_with_occupation["geometry"],
    )
    arrondissements_with_occupation["occupation"] = 100 * (
        arrondissements_with_occupation["num_bikes_available"]
        / arrondissements_with_occupation["capacity"]
    )
    return arrondissements_with_occupation


arrondissements_with_occupation = get_arrondissements_with_occupation(
    arrondissements, occupation_df_time
)
print(arrondissements_with_occupation.crs)
type(arrondissements_with_occupation)

In [ ]:
colorscale = ["yellow", "green"]
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"
color_prop = "occupation"
vmin = 0
vmax = 100
colorbar = dl.Colorbar(
    colorscale=colorscale,
    width=20,
    height=150,
    min=0,
    max=vmax,
    unit="usage %",
)
style_arrondissements = dict(fillOpacity=0.3)
style_occupation = dict(weight=1, dashArray="10", color="red", fillOpacity=0.3)
style_handle = assign(
    """
    function(feature, context){
        console.log(context.hideout);
        console.log(feature);
        const {min, max, colorscale, style, colorProp} = context.hideout;
        const csc = chroma.scale(colorscale).domain([min, max]);
        style.color = csc(feature.properties[colorProp]);
        return style;
    }
    """
)

app = dash.Dash(
    external_scripts=[chroma], external_stylesheets=[dbc.themes.BOOTSTRAP]
)

arrondissements_layer = dl.GeoJSON(
    data=None,
    id="arrondissements",
    options=dict(style=style_handle),
    hideout=dict(
        min=vmin,
        max=vmax,
        colorscale=colorscale,
        style=style_arrondissements,
        colorProp=color_prop,
    ),
    hoverStyle={"weight": 5},
    zoomToBoundsOnClick=True,
)


occupation_layer = dl.GeoJSON(
    data=None,
    id="occupation",
    options=dict(style=style_handle),
    hideout=dict(
        min=vmin,
        max=vmax,
        colorscale=colorscale,
        style=style_occupation,
        colorProp=color_prop,
    ),
    hoverStyle={"fillOpacity": 0.5},
    children=[dl.Popup(html.Div(id="occupation-popup"))],
)

app.layout = html.Div(
    [
        dcc.Store(id="arrondissements_data"),
        dcc.Store(id="occupation_data"),
        dcc.Store(id="polygon_arrondissement"),
        html.H1("Occupation des stations Vélib"),
        dbc.Input(value="07:40", id="time", type="text", placeholder="HH:MM"),
        dbc.Button("Changer l'heure", id="change-time"),
        html.Div(
            [dbc.Button("Reset map", id="reset")],
            style={"text-align": "right"},
        ),
        dl.Map(
            [
                dl.TileLayer(),
                arrondissements_layer,
                # quartier_paris_layer,
                # station_information_layer,
                occupation_layer,
                colorbar,
            ],
            center=[48.8566, 2.3522],
            zoom=12,
            style={"width": "100%", "height": "500px"},
            id="map",
        ),
        html.Div(id="graph"),
    ]
)


@app.callback(
    Output("arrondissements", "data"),
    Input("arrondissements_data", "data"),
)
def update_arrondissements_data(arrondissements_data):
    return arrondissements_data


@app.callback(
    Output("arrondissements_data", "data"),
    Input("occupation_data", "data"),
)
def inject_arrondissements_data(occupation_df_time):
    occupation_df_time = pd.DataFrame(occupation_df_time)
    arrondissements_with_occupation = get_arrondissements_with_occupation(
        arrondissements, occupation_df_time
    )
    arrondissements_with_occupation[
        "tooltip"
    ] = arrondissements_with_occupation["l_ar"]
    return json.loads(arrondissements_with_occupation.to_json())


@app.callback(
    Output("occupation_data", "data"),
    Input("change-time", "n_clicks"),
    State("time", "value"),
)
def update_occupation_data(n_clicks, time_str):
    if time_str is None:
        return occupation_df_time
    hour, minute = extract_hour_minute(time_str)
    occupation_df_time = filter_time(occupation_df, hour, minute)
    return occupation_df_time.to_dict(orient="records")


@app.callback(
    Output("polygon_arrondissement", "data"),
    Input("arrondissements", "clickData"),
)
def update_click_arrondissements(feature):
    if feature is None:
        return None
    return feature["geometry"]["coordinates"][0]


@app.callback(
    Output("occupation", "data"),
    Input("reset", "n_clicks"),
    Input("occupation_data", "data"),
    Input("polygon_arrondissement", "data"),
)
def update_occupation_layer(
    reset_n_clicks, occupation_df_time, polygon_arrondissement
):
    ctx = dash.callback_context
    if not ctx.triggered:
        return None

    prop_id = ctx.triggered[0]["prop_id"]
    if "reset" in prop_id:
        return None
    else:
        polygon = Polygon(polygon_arrondissement)
        occupation_df_time = pd.DataFrame(occupation_df_time)
        station_occupation_time = join_occupation_and_station_information(
            occupation_df_time,
            gpd.GeoDataFrame(
                station_information,
                geometry=station_catchment_area,
                crs="EPSG:4326",
            ),
        )
        intersection = station_occupation_time.intersection(polygon)
        station_intersection = station_occupation_time[
            ~intersection.is_empty
        ].copy()
        station_intersection.geometry = intersection[~intersection.is_empty]

        data = json.loads(station_intersection.to_json())
        return data


@app.callback(
    Output("occupation-popup", "children"),
    Input("occupation", "clickData"),
)
def update_popup(feature):
    if feature is None:
        return None
    _html = [
        html.H4(f"Station: {feature['properties']['name']}"),
        html.P(
            f"Nombre de vélos disponibles : {feature['properties']['num_bikes_available']}/{feature['properties']['capacity']}"
        ),
        html.P(f"Occupation: {feature['properties']['occupation']:.2f}%"),
    ]
    return _html


@app.callback(
    Output("graph", "children"),
    Input("occupation", "clickData"),
)
def update_graph(feature):
    if feature is None:
        return None
    station_name = feature["properties"]["name"]
    occupation_df_station = occupation_df[
        occupation_df["name"] == station_name
    ]
    fig = px.line(
        occupation_df_station,
        x="datetime",
        y="num_bikes_available",
        title=f"Station {station_name}",
    )
    fig.add_hline(
        occupation_df_station["capacity"].iloc[0],
        line_dash="dash",
        annotation_text="Capacité maximale",  # Add the annotation text
        annotation_position="bottom right",
    )
    graph = dcc.Graph(figure=fig)
    return graph


app.run()

In [ ]:
# quartier_paris_layer = dl.GeoJSON(
#     data=None,
#     id="quartier_paris",
#     options=dict(style=dict(color="red", fillOpacity=0)),
#     hoverStyle={"weight": 5},
# )

# station_information_layer = dl.GeoJSON(
#     data=json.loads(station_information.to_json()),
#     id="station_information",
#     options=dict(pointToLayer=point_to_layer),
#     cluster=True,
#     superClusterOptions=dict(radius=150),
#     zoomToBoundsOnClick=True,
#     hoverStyle={"stroke": True, "width": 5},
# )


# @app.callback(
#     Output("quartier_paris", "data"),
#     Input("arrondissements", "clickData"),
#     Input("reset", "n_clicks"),
# )
# def update_feature_info(feature, reset_n_clicks):
#     ctx = dash.callback_context
#     if not ctx.triggered:
#         return None

#     prop_id = ctx.triggered[0]["prop_id"]
#     if "reset-button" in prop_id:
#         return None
#     elif "arrondissements.clickData" in prop_id:
#         return json.loads(quartier_paris[quartier_paris["c_ar"] == feature["properties"]["c_ar"]].to_json())